In [1]:
import pandas as pd
from joblib import Parallel, delayed
import numpy as np
import contextlib
import joblib
from tqdm import tqdm

# calculate song ranking

In [2]:
def calc_rank(df):
    df['rank'] = 0
    df['rank'] = df['rank'].astype(float)
    #popularity
    max_pop = df['popularity'].max()
    min_pop = df['popularity'].min()
    #listeners
    max_listeners = df['Listeners'].max()
    min_listeners = df['Listeners'].min()
    #playcount
    max_playcount = df['Playcount'].max()
    min_playcount = df['Playcount'].min()
    #chart
    #(101-peak)*weeks
    df['chart_rank'] = 0
    for i in range(len(df)):
        try:
            df['chart_rank'].iloc[i] = (101 - int(df['chart_peak'].iloc[i])) * int(df['chart_weeks'].iloc[i])
        except:
            df['chart_rank'].iloc[i] = 0
    max_chart = df['chart_rank'].max()
    min_chart = df['chart_rank'].min()

    for i in range(len(df)):
        #progress
        if i % 1000 == 0:
            print(round(i/len(df)*100))
        try:
            #min max scaling
            # and 0% popularity + listeners, playcount, charts each 20%
            df['rank'].iloc[i] = (int(df['popularity'].iloc[i]) - min_pop) / (max_pop - min_pop) * 0.4 + \
                                 (int(df['Listeners'].iloc[i]) - min_listeners) / (max_listeners - min_listeners) * 0.2 + \
                                 (int(df['Playcount'].iloc[i]) - min_playcount) / (max_playcount - min_playcount) * 0.2 + \
                                 (int(df['chart_rank'].iloc[i]) - min_chart) / (max_chart - min_chart) * 0.2
        except:
            df['rank'].iloc[i] = 0
    return df

In [ ]:
df = pd.read_csv('data_cut_missing_lyrics.csv')
df = calc_rank(df)
df.head()

In [4]:
#sort by rank
df = df.sort_values(by=['rank'], ascending=False)
df.head()

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
835,GBAFL8600048,"['alt-rock', 'alternative', 'indie', 'indie-po...","sea, swallow me","cocteau twins,harold budd",The Moon and the Melodies,1986-11-10,day,spotify:track:7vPHiwgCOAjSBRjAirbMvS,7vPHiwgCOAjSBRjAirbMvS,NaN,...,457980,5130122,"['dream pop', 'shoegaze', 'alternative rock', ...","sea, swallow me",https://www.musixmatch.com/lyrics/Cocteau-Twin...,no lyrics,0,0,0.800000,0
836,GBAFL8400054,"['alt-rock', 'alternative', 'indie', 'indie-po...",lorelei,cocteau twins,Treasure,1984-11-01,day,spotify:track:1ZnXuijpwEBjhNxpEnb8WH,1ZnXuijpwEBjhNxpEnb8WH,NaN,...,380071,2646044,"['dream pop', 'ethereal', '80s', 'alternative'...",lorelei,https://www.musixmatch.com/lyrics/Cocteau-Twin...,no lyrics,0,0,0.585526,0
3705,USSM19932782,"['blues', 'hard-rock', 'rock']",little wing,stevie ray vaughan,Couldn't Stand The Weather (Legacy Edition),1984,year,spotify:track:5XNpdKmlLJPUbwKQceX2tW,5XNpdKmlLJPUbwKQceX2tW,NaN,...,231450,937612,"['blues', 'blues rock', 'guitar', 'guitar virt...",little wing,no url,no lyrics,0,0,0.471902,0
321,GBAAN7390003,"['reggae', 'ska']",i shot the sheriff,the wailers,Legend (Deluxe Edition),1984-05-01,day,spotify:track:71zvW0WZ0bap9gphp7Jhgp,71zvW0WZ0bap9gphp7Jhgp,NaN,...,223353,852512,"['reggae', '70s', '1973', 'roots reggae', 'jam...",i shot the sheriff,https://genius.com/The-wailers-i-shot-the-sher...,no lyrics,0,0,0.435196,0
47,GBAAA0500103,['ambient'],an ending (ascent) - remastered 2005,brian eno,Apollo,1983-07-01,day,spotify:track:1vgSaC0BPlL6LEm4Xsx59J,1vgSaC0BPlL6LEm4Xsx59J,NaN,...,147710,846115,"['electronic', 'beautiful', 'contemplative']",an ending,https://genius.com/Brian-eno-an-ending-ascent-...,no lyrics,0,0,0.425787,0


In [5]:
df.tail()

,isrc,genres,name,artists,album,release_date,release_date_precision,uri,spotify_id,chart_power,...,Listeners,Playcount,Tags,clear_name,lyrics_url,lyrics,chart_peak,chart_weeks,rank,chart_rank
8469,QM6P41496250,['new-age'],fairyland,medwyn goodall,Kindred Spirits,1987-01-01,day,spotify:track:7FCbqQuu3lEhBP7uCrEUtY,7FCbqQuu3lEhBP7uCrEUtY,NaN,...,212,1611,"['genre: new age instrumental', 'radio-dawn-fa...",fairyland,no url,no lyrics,0,0,0.000072,0
8295,DEZ650603642,"['disco', 'party']",banane du bist fällig,klaus & klaus,Steht da Ein Pferd Auf'm Flur,1984,year,spotify:track:2kpcy2vB6aEuWnppu3qJg1,2kpcy2vB6aEuWnppu3qJg1,NaN,...,120,2196,[],banane du bist fällig,no url,no lyrics,0,0,0.000054,0
8515,QM6P41496908,"['hardcore', 'punk']",lying dead on the phone,flower leperds,The Original Group (Remastered),1985,year,spotify:track:1a9I9o1BpJJxb7CDuvOlY8,1a9I9o1BpJJxb7CDuvOlY8,NaN,...,223,1036,[],lying dead on the phone,no url,no lyrics,0,0,0.000054,0
8300,GB5KW2003027,"['reggae', 'rock']",imagine,janet kay,Sweet Surrender,1989-04-10,day,spotify:track:5ByKOkY5EKg2hWmFYzbCcR,5ByKOkY5EKg2hWmFYzbCcR,NaN,...,225,1005,[],imagine,no url,no lyrics,0,0,0.000054,0
3660,SEPQM0611416,"['pop', 'swedish']",minnet,kikki danielsson,Kikki,1982,year,spotify:track:1CXkza9lVfPHPYUBkbLQK6,1CXkza9lVfPHPYUBkbLQK6,NaN,...,141,1766,"['svensk country taggradio', 'kikki danielsson...",minnet,https://www.musixmatch.com/lyrics/Kikki-Daniel...,no lyrics,0,0,0.000047,0


In [51]:
df['rank'].describe()

count    45210.000000
mean         0.088901
std          0.072490
min          0.000007
25%          0.034509
50%          0.073248
75%          0.124750
max          0.702498
Name: rank, dtype: float64

In [53]:
#store the data
# df.to_csv('final_dataset.csv', index=False)